# Q3.

#### At this point, you understand the data quite well. Propose and carry out additional analysis using the dataset given. Please be sure to justify why this additional analysis is useful and interesting.

- Build a classifier to label readings (0-suspicious, 1-normal)
    - can just use rule-based
- Predict future malfunction timestamp
- Predict peak consumption & its period

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import statistics

In [2]:
#reading in given dataset
df = pd.read_csv("dataport-export_gas_oct2015-mar2016.csv")

In [3]:
#sort by dataid & timestamp
dfSorted = df.sort_values(by=['dataid', 'localminute'])

In [4]:
dfSorted = dfSorted.reset_index(drop=True)

In [5]:
#compile a list of 157 unique meter IDs
all_ids = sorted(dfSorted['dataid'].unique())

In [6]:
#function to identify meter IDs with duplicate readings
def checkDuplicates(df, id):
    data = df.loc[df['dataid'] == id]
    boolList = list(data.localminute.duplicated())
    for i in boolList:
        if(i == True):
            return True
    return False
# print(checkDuplicates(dfSorted, 35))   #check specific meter ID

#Visualize all meter IDs with duplicates using above function
hasDuplicates = []
for i in all_ids:
    if(checkDuplicates(dfSorted, i) == True):
        hasDuplicates.append(i)
# print("Meter IDs with duplicate readings: ", hasDuplicates)
# print("Indexes of these meter IDs: ", all_ids.index(4031), all_ids.index(8890))

In [7]:
#create an overall df with resampled localminute in hours
startDate = datetime(2015, 10, 1)
endDate = datetime(2016, 4, 1)
hours = pd.date_range(startDate, endDate, freq='H', closed='left')   #hourly freq

#overall df
dfResampled = pd.DataFrame()
dfResampled['localminute'] = hours

#concat cols of meter readings to overall df
for j in range(len(all_ids)):
    data = dfSorted.loc[dfSorted["dataid"] ==  all_ids[j]]   #select rows of specified meter ID
    data = data.drop_duplicates(subset=['localminute'])   #drop duplicate readings
    
    form = '%Y-%m-%d %H:%M:%S'
    timeList = []
    for i in range(len(data)):
        time = datetime.strptime(data.localminute.values[i][:19], form)   #extract datetime up to seconds
        timeList.append(time)
    data.index = timeList
    data = data.drop(columns = 'localminute')    
    meter_hourly = data.resample('H').max()   #max reading for each resampled hour
    #meter_hourly = meter_hourly.fillna(method='ffill')   #forward fill NaN cells   
    
    #create new df containing just 1 col of interpolated readings for that meter ID
    dfTemp = pd.DataFrame()
    dfTemp[all_ids[j]] = list(meter_hourly.meter_value.values)   
    dfResampled = pd.concat([dfResampled, dfTemp], axis=1)   #concat new col of interpolated readings to overall df
    
dfResampled = dfResampled.fillna(method='ffill')   #forward fill all NaN values    
dfResampled

,localminute,35,44,77,94,114,187,222,252,370,...,9474,9600,9620,9631,9639,9729,9766,9849,9956,9982
0,2015-10-01 00:00:00,93470.0,165674.0,58832.0,116642.0,128294.0,263272.0,612262.0,329214.0,87880.0,...,206436.0,121174.0,432688.0,108348.0,211656.0,122186.0,158512.0,33632.0,95860.0,105946.0
1,2015-10-01 01:00:00,93470.0,165674.0,58832.0,116644.0,128296.0,263272.0,612262.0,329214.0,87880.0,...,206438.0,121180.0,432688.0,108350.0,211658.0,122186.0,158512.0,33632.0,95860.0,105946.0
2,2015-10-01 02:00:00,93470.0,165674.0,58832.0,116644.0,128296.0,263272.0,612262.0,329216.0,87880.0,...,206438.0,121188.0,432688.0,108350.0,211658.0,122186.0,158512.0,33632.0,95860.0,105946.0
3,2015-10-01 03:00:00,93470.0,165684.0,58832.0,116644.0,128296.0,263272.0,612262.0,329216.0,87880.0,...,206438.0,121188.0,432688.0,108350.0,211658.0,122188.0,158512.0,33632.0,95860.0,105946.0
4,2015-10-01 04:00:00,93470.0,165684.0,58832.0,116644.0,128296.0,263272.0,612264.0,329216.0,87880.0,...,206438.0,121188.0,432688.0,108350.0,211658.0,122188.0,158512.0,33632.0,95860.0,105946.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,2016-03-31 19:00:00,104682.0,175070.0,64652.0,140358.0,145298.0,285492.0,677486.0,351332.0,100794.0,...,234268.0,124540.0,466648.0,124478.0,255418.0,138144.0,179902.0,39292.0,107042.0,119924.0
4388,2016-03-31 20:00:00,104682.0,175070.0,64652.0,140358.0,145298.0,285492.0,677486.0,351332.0,100794.0,...,234268.0,124540.0,466648.0,124478.0,255418.0,138144.0,179902.0,39292.0,107042.0,119924.0
4389,2016-03-31 21:00:00,104684.0,175070.0,64652.0,140358.0,145298.0,285492.0,677486.0,351334.0,100816.0,...,234268.0,124540.0,466648.0,124488.0,255418.0,138144.0,179902.0,39292.0,107042.0,119924.0
4390,2016-03-31 22:00:00,104690.0,175070.0,64652.0,140358.0,145298.0,285492.0,677486.0,351334.0,100816.0,...,234268.0,124540.0,466648.0,124488.0,255418.0,138146.0,179902.0,39292.0,107042.0,119924.0


In [8]:
dfResampled[4874]

0       307502.0
1       307502.0
2       307502.0
3       307502.0
4       307502.0
          ...   
4387    314488.0
4388    314488.0
4389    314488.0
4390    314488.0
4391    314488.0
Name: 4874, Length: 4392, dtype: float64

### transpose

In [9]:
dfResampled2 = dfResampled.copy()

In [10]:
#transpose the df
dfResampled2 = dfResampled2.T

#set first row as df header
newHeader = dfResampled2.iloc[0] 
dfResampled2 = dfResampled2.iloc[1:] 
dfResampled2.columns = newHeader
dfResampled2

localminute,2015-10-01 00:00:00,2015-10-01 01:00:00,2015-10-01 02:00:00,2015-10-01 03:00:00,2015-10-01 04:00:00,2015-10-01 05:00:00,2015-10-01 06:00:00,2015-10-01 07:00:00,2015-10-01 08:00:00,2015-10-01 09:00:00,...,2016-03-31 14:00:00,2016-03-31 15:00:00,2016-03-31 16:00:00,2016-03-31 17:00:00,2016-03-31 18:00:00,2016-03-31 19:00:00,2016-03-31 20:00:00,2016-03-31 21:00:00,2016-03-31 22:00:00,2016-03-31 23:00:00
35,93470.0,93470.0,93470.0,93470.0,93470.0,93470.0,93472.0,93472.0,93472.0,93472.0,...,104670.0,104672.0,104672.0,104672.0,104672.0,104682.0,104682.0,104684.0,104690.0,104692.0
44,165674.0,165674.0,165674.0,165684.0,165684.0,165686.0,165686.0,165686.0,165686.0,165688.0,...,175070.0,175070.0,175070.0,175070.0,175070.0,175070.0,175070.0,175070.0,175070.0,175070.0
77,58832.0,58832.0,58832.0,58832.0,58832.0,58834.0,58840.0,58840.0,58846.0,58846.0,...,64642.0,64646.0,64646.0,64646.0,64646.0,64652.0,64652.0,64652.0,64652.0,64652.0
94,116642.0,116644.0,116644.0,116644.0,116644.0,116644.0,116644.0,116646.0,116646.0,116646.0,...,140356.0,140356.0,140356.0,140358.0,140358.0,140358.0,140358.0,140358.0,140358.0,140358.0
114,128294.0,128296.0,128296.0,128296.0,128296.0,128296.0,128298.0,128298.0,128298.0,128308.0,...,145298.0,145298.0,145298.0,145298.0,145298.0,145298.0,145298.0,145298.0,145298.0,145298.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9729,122186.0,122186.0,122186.0,122188.0,122188.0,122188.0,122188.0,122190.0,122196.0,122196.0,...,138136.0,138142.0,138142.0,138144.0,138144.0,138144.0,138144.0,138144.0,138146.0,138146.0
9766,158512.0,158512.0,158512.0,158512.0,158512.0,158512.0,158512.0,158512.0,158512.0,158512.0,...,179902.0,179902.0,179902.0,179902.0,179902.0,179902.0,179902.0,179902.0,179902.0,179902.0
9849,33632.0,33632.0,33632.0,33632.0,33632.0,33638.0,33638.0,33638.0,33638.0,33638.0,...,39292.0,39292.0,39292.0,39292.0,39292.0,39292.0,39292.0,39292.0,39292.0,39292.0
9956,95860.0,95860.0,95860.0,95860.0,95860.0,95862.0,95862.0,95868.0,95870.0,95870.0,...,107042.0,107042.0,107042.0,107042.0,107042.0,107042.0,107042.0,107042.0,107042.0,107042.0


In [11]:
#normalize
from sklearn.preprocessing import normalize
dfNormalized = normalize(dfResampled2)
dfNormalized = pd.DataFrame(dfNormalized, columns=dfResampled2.columns)
dfNormalized

localminute,2015-10-01 00:00:00,2015-10-01 01:00:00,2015-10-01 02:00:00,2015-10-01 03:00:00,2015-10-01 04:00:00,2015-10-01 05:00:00,2015-10-01 06:00:00,2015-10-01 07:00:00,2015-10-01 08:00:00,2015-10-01 09:00:00,...,2016-03-31 14:00:00,2016-03-31 15:00:00,2016-03-31 16:00:00,2016-03-31 17:00:00,2016-03-31 18:00:00,2016-03-31 19:00:00,2016-03-31 20:00:00,2016-03-31 21:00:00,2016-03-31 22:00:00,2016-03-31 23:00:00
0,0.014276,0.014276,0.014276,0.014276,0.014276,0.014276,0.014277,0.014277,0.014277,0.014277,...,0.015987,0.015987,0.015987,0.015987,0.015987,0.015989,0.015989,0.015989,0.015990,0.015990
1,0.014621,0.014621,0.014621,0.014622,0.014622,0.014622,0.014622,0.014622,0.014622,0.014622,...,0.015450,0.015450,0.015450,0.015450,0.015450,0.015450,0.015450,0.015450,0.015450,0.015450
2,0.014420,0.014420,0.014420,0.014420,0.014420,0.014420,0.014422,0.014422,0.014423,0.014423,...,0.015844,0.015845,0.015845,0.015845,0.015845,0.015846,0.015846,0.015846,0.015846,0.015846
3,0.013797,0.013797,0.013797,0.013797,0.013797,0.013797,0.013797,0.013797,0.013797,0.013797,...,0.016602,0.016602,0.016602,0.016602,0.016602,0.016602,0.016602,0.016602,0.016602,0.016602
4,0.014186,0.014186,0.014186,0.014186,0.014186,0.014186,0.014187,0.014187,0.014187,0.014188,...,0.016066,0.016066,0.016066,0.016066,0.016066,0.016066,0.016066,0.016066,0.016066,0.016066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.014230,0.014230,0.014230,0.014230,0.014230,0.014230,0.014230,0.014230,0.014231,0.014231,...,0.016087,0.016088,0.016088,0.016088,0.016088,0.016088,0.016088,0.016088,0.016088,0.016088
153,0.014188,0.014188,0.014188,0.014188,0.014188,0.014188,0.014188,0.014188,0.014188,0.014188,...,0.016103,0.016103,0.016103,0.016103,0.016103,0.016103,0.016103,0.016103,0.016103,0.016103
154,0.013950,0.013950,0.013950,0.013950,0.013950,0.013953,0.013953,0.013953,0.013953,0.013953,...,0.016298,0.016298,0.016298,0.016298,0.016298,0.016298,0.016298,0.016298,0.016298,0.016298
155,0.014271,0.014271,0.014271,0.014271,0.014271,0.014271,0.014271,0.014272,0.014272,0.014272,...,0.015935,0.015935,0.015935,0.015935,0.015935,0.015935,0.015935,0.015935,0.015935,0.015935
